In [1]:
import json
from konlpy.tag import Kkma
from konlpy.utils import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC
from sklearn.metrics import accuracy_score,mean_squared_error
import numpy as np

In [2]:
REVIEWS = "cs_reviews"

In [35]:
f = open(""+REVIEWS+".txt","r",encoding="utf-8").readlines()

In [11]:
g = open(REVIEWS+"_nouns.txt","r",encoding="utf-8").readlines()

In [12]:
kkma = Kkma()

In [36]:
# with open("../reviews_in_one_txt/" + REVIEWS+"_nouns.txt","w",encoding="utf-8") as out:
#     for line in f[:5]:
#         course = json.loads(line)
#         nouns = kkma.nouns(course["review_text"])
# #         print(nouns)
#         out.write(" ".join(nouns)+"\n")
        

In [22]:
concat_nouns_list = []
mp = {"A":5,"B":4,"C":3,"D":2,"F":1,}
labels = []
for i in range(len(f)):
    line = f[i]
    course = json.loads(line)
    if course["널널"] != "?" and len(g[i])>1:
        concat_nouns = g[i]
        concat_nouns_list.append(concat_nouns)
        labels.append(mp[course["널널"]])

In [23]:
labels = np.array(labels)
concat_nouns_list = np.array(concat_nouns_list)
data_train, data_test, y_train, y_test = train_test_split(concat_nouns_list,
                                    labels,test_size=0.1,random_state=42)

In [24]:
len(labels)

481

In [25]:
vectorizer = TfidfVectorizer()

In [26]:
X_train = vectorizer.fit_transform(data_train)
X_test = vectorizer.transform(data_test)

In [27]:
print(X_train.shape)

(432, 3248)


In [28]:
print(X_test.shape)

(49, 3248)


In [29]:
clf = LinearSVC()
clf.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [30]:
predictions = clf.predict(X_test)
accuracy_score(y_test,predictions)

0.3673469387755102

In [31]:
mean_squared_error(y_test,predictions)

1.9795918367346939

In [32]:
predictions

array([5, 4, 5, 5, 3, 5, 4, 5, 5, 5, 3, 1, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3,
       5, 5, 4, 5, 5, 4, 5, 3, 5, 5, 5, 1, 5, 4, 5, 3, 5, 4, 3, 5, 5, 4,
       5, 4, 5, 4, 3])

In [33]:
y_test

array([5, 5, 4, 4, 1, 5, 2, 4, 3, 1, 4, 1, 5, 4, 3, 3, 4, 3, 4, 5, 2, 5,
       5, 5, 4, 4, 5, 5, 4, 1, 3, 4, 5, 1, 3, 5, 3, 5, 3, 4, 3, 1, 4, 4,
       5, 3, 5, 2, 4])